In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from functools import partial

# MODEL 1.0

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
X_train, y_train = X_train_full[5000:], y_train_full[5000:]
X_val, y_val = X_train_full[:5000], y_train_full[:5000]
X_train = X_train / 255.
X_val = X_val / 255.
X_test = X_test / 255.

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_train

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, 
                              padding="SAME", input_shape=[28,28,1]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPooling2D(pool_size=2))

model.add(keras.layers.Conv2D(filters=64, kernel_size=3, 
                              padding="same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=3, 
                              padding="same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPooling2D(pool_size=2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation="softmax"))

In [5]:
# model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
# model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

In [6]:
# model.evaluate(X_test,y_test)

# MODEL 2.0

In [7]:
keras.backend.clear_session()

In [8]:
model_2 = keras.models.Sequential([
#     conv layer 1
    keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding="same",
                       input_shape=[28,28,1]),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    
    keras.layers.MaxPool2D(pool_size=2, padding="same"),
    keras.layers.Dropout(rate=0.3),
#     conv layer 2
    keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    
    keras.layers.MaxPool2D(pool_size=4, padding="same"),
    keras.layers.Dropout(rate=0.3),
    keras.layers.Flatten(),
    
#     hidden layer 3
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(rate=0.4),
    
    keras.layers.Dense(10, activation="softmax")
    
    
])

In [9]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
)

# Reduce learning rate when validation loss plateaus
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5
)

In [10]:
# model_2.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
# model_2.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

In [11]:
# model_2.evaluate(X_test,y_test)

In [12]:
# y_pred = np.argmax(model.predict(X_test), axis=1)

# MODEL 3.0 WITH AUGMENTED DATA



In [13]:
keras.backend.clear_session()

In [14]:
from scipy import ndimage
import random

def data_augmentation(images, labels):
    aug_images = []
    aug_labels = []
    
    for image, label in zip(images, labels):
        aug_images.append(image)
        aug_labels.append(label)
        
        # Background image for filling empty pixel
        bg_value = np.median(image)
        for _ in range(4):
            # Rotation
            rot_image = ndimage.rotate(image, angle=random.randint(-15, 15), 
                                       reshape=False, cval=bg_value)
            # Shift
            shift_image = ndimage.shift(rot_image, shift=np.random.randint(-2, 2, 2), 
                                        cval=bg_value)
            
            aug_images.append(shift_image)
            aug_labels.append(label)
    aug_images = np.array(aug_images)
    aug_labels = np.array(aug_labels)
    return aug_images, aug_labels

In [15]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, y_train = data_augmentation(X_train, y_train)

# Convert numpy float type and normalize it
X_train = X_train.astype(np.float32) / 255.
X_test = X_test.astype(np.float32) / 255.

# Convert it to 4D array (or we can use np.expand_dims for dimension expansion)
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

# Build dataset pipeline
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=500000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [16]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
)

# Reduce learning rate when validation loss plateaus
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5
)

In [17]:
model_2.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
model_2.fit(train_ds, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

Epoch 1/10
9375/9375 [==============================] - 409s 43ms/step - loss: 0.1897 - accuracy: 0.9416 - val_loss: 0.0298 - val_accuracy: 0.9918 - lr: 0.0010
Epoch 2/10
9375/9375 [==============================] - 416s 44ms/step - loss: 0.0738 - accuracy: 0.9780 - val_loss: 0.0154 - val_accuracy: 0.9944 - lr: 0.0010
Epoch 3/10
9375/9375 [==============================] - 423s 45ms/step - loss: 0.0551 - accuracy: 0.9837 - val_loss: 0.0129 - val_accuracy: 0.9966 - lr: 0.0010
Epoch 4/10
9375/9375 [==============================] - 420s 45ms/step - loss: 0.0458 - accuracy: 0.9864 - val_loss: 0.0105 - val_accuracy: 0.9958 - lr: 0.0010
Epoch 5/10
9375/9375 [==============================] - 419s 45ms/step - loss: 0.0407 - accuracy: 0.9880 - val_loss: 0.0090 - val_accuracy: 0.9974 - lr: 0.0010
Epoch 6/10
9375/9375 [==============================] - 419s 45ms/step - loss: 0.0374 - accuracy: 0.9889 - val_loss: 0.0058 - val_accuracy: 0.9980 - lr: 0.0010
Epoch 7/10
9375/9375 [==================

In [18]:
model_2.evaluate(test_ds)

313/313 [==============================] - 4s 11ms/step - loss: 0.0128 - accuracy: 0.9961


[0.012812662869691849, 0.9961000084877014]